Voice-Activated Personal Assistant Project
------------------------------------------
This project develops a voice-activated personal assistant using Python. It combines speech recognition and text-to-speech for interactive communication. Key features include:

- Voice command recognition for initiating various tasks.
- Integration with OpenAI's GPT for a responsive chat mode.
- Ability to play music from YouTube based on voice requests.
- Real-time weather updates using WeatherAPI.
- Customizable settings for voice adjustments.

This assistant is designed to assist users in daily tasks through voice commands, enhancing user experience with AI-powered conversational capabilities.

# Import the dependencies

In [ ]:
import speech_recognition as sr
import pyttsx3
import time
import requests
import json
import vlc
from pytube import YouTube  
import os
from openai import OpenAI

# Initialize the variable

In [ ]:
client = OpenAI()
listeningToCommand = True;
engine = pyttsx3.init('sapi5')

volume = engine.getProperty('volume')
engine.setProperty('volume', volume+0.25)

rate = engine.getProperty('rate')
engine.setProperty('rate', rate-10)

# Declaring the function

The function to output the audio to user

In [ ]:
def speak(scripts):
    engine.say(scripts)
    engine.runAndWait()


The function to get audio input from user

In [ ]:
def audioInput(pauseThreshold=0.8, energyThreshold=4000, mode='normal'):
    # obtain audio from the microphone
    r = sr.Recognizer()
    r.energy_threshold = energyThreshold
    r.pause_threshold = pauseThreshold
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print("Listening...")
        audio = r.listen(source)
    
    # recognize speech using Google Speech Recognition
    try:
    # for testing purposes, we're just using the default API key
    # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
    # instead of `r.recognize_google(audio)`
        print("Google Speech Recognition thinks you said " + r.recognize_google(audio))
        return r.recognize_google(audio)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        if mode == 'normal':
            speak("Please try again, I can't understand you")
        return "try again"
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        if mode == 'normal':
            speak("Please try again, I can't understand you")
        return "try again"
    
    

The function to determine to stop or continue the personal assistant

In [ ]:
def action(audioInput):
    assistant = audioInput()
    
    if 'billy' in assistant.lower():
        return 'billy'
    if 'stop' in assistant.lower():
        return 'stop'

The function to call the helper or stop the program

In [ ]:
def personalAssistant(choice):
    if choice == 'billy':
        speak('Hi! Im billy, your personal assistant. How can I help you?')
        helper()
    if choice == 'stop':
        global listeningToCommand
        listeningToCommand = False

The function that call determine the request of the user

In [ ]:
def helper():
    request = audioInput()
    
    if 'chat' in request.lower():
        speak("Entering Chat Mode")
        chatMode()
    
    if 'play' in request.lower():
        url = "https://youtube-search-and-download.p.rapidapi.com/search"
        headers = {
            "X-RapidAPI-Key": "62ab3cbf75msh1c39d509569cbbcp1f014bjsnc70a6e8a4a57",
            "X-RapidAPI-Host": "youtube-search-and-download.p.rapidapi.com"
        }
        
        links = 'https://www.youtube.com/watch?v='
        songsName = request.partition("play")[2]
        response = requests.get(url, headers=headers, params={"query":songsName, "type":"v"})
        videoId = response.json()['contents'][0]['video']['videoId']
        videoUrl = links + videoId
        print(videoUrl)
        speak("Please wait as I'm searching for the song")
        playSong(videoUrl)
    
    if 'weather' in request.lower():
        speak("Please wait while I'm look up the weather")
        weather = getWeather()
        scripts = "The weather in Madison right now is ", weather, " degree Celcius!"
        speak(scripts)
        
    
    if 'stop' in request.lower():
        global listeningToCommand
        listeningToCommand = False

The function that handle chatMode request.

In [ ]:
def chatMode():
    keep_running= True
    
    messagesList=[
                {"role": "system", "content": "You are a helpful assistant for a student named Muhammad Irfan that lived in madison wisconsin united states and studying computer science and data science in University of Wisconsin Madison. You will always response in 50 words or less only"},
            ]
    
    
    while keep_running:
        request = audioInput()
        
        messagesList.append({"role": "user", "content": request})
    
        if 'stop' in request:
            keep_running = False
            break
    
        speak("Please wait while I'm thinking...")
    
        textResponse = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messagesList
        )
    
        text = textResponse.choices[0].message.content
        messagesList.append({"role": "assistant", "content": text})
        speak(text)
        
        speak("Do you need anything else? if not say stop to exit chat mode")
        
    
    
    return
    
    
    
    

The function that handle play songs request

In [ ]:
def playSong(playurl):
    currPath = r"C:\Users\16089\Documents\Home Assistant"
    
    yt = YouTube(playurl)
    length = yt.length
    name = yt.title
    yt = yt.streams.get_audio_only()
    yt.download(filename="song.mp4")
    
    
    timeout = time.time() + length + 3
    
    speak('Playing ' + name)
    media = vlc.MediaPlayer("song.mp4")
    media.play()
    
    
    
    while True:
        stopCommand = audioInput(mode='playingSong')
        if time.time() > timeout:
            media.stop()
            break
        if 'stop' in stopCommand:
            speak("Stopping the song")
            media.stop()
            break


The function that handle get weather request from user

In [ ]:
def getWeather():
    url = "https://weatherapi-com.p.rapidapi.com/current.json"

    headers = {
        "X-RapidAPI-Key": "62ab3cbf75msh1c39d509569cbbcp1f014bjsnc70a6e8a4a57",
        "X-RapidAPI-Host": "weatherapi-com.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params={"q":"43.073051,-89.401230"})

    weather = str(response.json()['current']['feelslike_c'])
    
    if '-' in weather:
        weather = weather.replace('-', 'negative ')
        
    return weather

Main function that run until user stops the program

In [ ]:
def run():
    while True:
        speak("Say billy to start")
        choice = action(audioInput)
        personalAssistant(choice)
    
        if not listeningToCommand:
            break
    
    return

# Running the program

In [ ]:
run()